In [3]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import haversine as hv
import numpy as np
import ipywidgets
from math import exp

In [6]:
# Read in 
location = pd.read_csv("/home/mavis22/work/Final project/Part 1/location.txt", delimiter="\t", names=["longitude","latitude","a","b"])
del location["a"], location["b"]
# From examples_week10.ipynb (IS590DV Class example)

In [7]:
# Tohoku : 142.369, 38.322
Tohoku=(142.369, 38.322)

In [8]:
# Calculate the distances
distance=[]
for row in location.iterrows():
    station = (row[1][0],row[1][1])
    distance.append(hv.haversine(Tohoku,station))

In [9]:
# Add distance column to location DataFrame
location["distance"]=distance

In [10]:
location["order"]=location.index

In [11]:
# Sort by distacnes (acsending)
location = location.sort_values(by=['distance'])

In [12]:
order=list(location["order"])

In [14]:
# Read in detector data
detector=pd.read_csv("/home/mavis22/work/data-readonly/transportable_array/data_tohoku_norm_transpose.csv",names=np.arange(0,438), index_col=False)

In [15]:
ordered_detector = detector[order[0]]
for each in order[1:]:
    ordered_detector = pd.concat([ordered_detector,detector[each]],axis=1)

In [16]:
# Add time range column, from week 10 example
time = pd.date_range("2:46PM", "6:46PM", freq="1s")
time -= time[0]
ordered_detector["time"] = time
ordered_detector.set_index("time", inplace=True)

In [17]:
ordered_detector.head()

,211,193,228,244,194,165,257,151,288,122,...,63,50,121,359,375,242,286,269,304,287
time,,,,,,,,,,,,,,,,,,,,,
00:00:00,-0.001273,0.002861,0.000524,-0.000501,0.000636,0.000501,-0.000090,-0.000892,0.000211,0.000612,...,NaN,-0.000249,-0.000137,0.000077,-0.000191,0.002128,0.000049,0.000025,-0.000400,0.000063
00:00:01,-0.001865,0.003157,0.000142,-0.000291,0.000863,0.000159,-0.000125,-0.001083,0.000425,0.000394,...,NaN,-0.000256,-0.000088,0.000217,0.000294,0.001515,0.000009,-0.000152,0.000151,-0.000126
00:00:02,-0.002425,0.003419,-0.000229,-0.000084,0.001077,-0.000176,-0.000157,-0.001258,0.000629,0.000177,...,NaN,-0.000261,-0.000039,0.000351,0.000764,0.000905,-0.000029,-0.000323,0.000685,-0.000309
00:00:03,-0.002937,0.003642,-0.000577,0.000114,0.001272,-0.000493,-0.000186,-0.001414,0.000818,-0.000030,...,NaN,-0.000263,0.000008,0.000476,0.001206,0.000318,-0.000067,-0.000482,0.001187,-0.000480
00:00:04,-0.003392,0.003827,-0.000895,0.000298,0.001443,-0.000787,-0.000212,-0.001546,0.000985,-0.000221,...,NaN,-0.000263,0.000052,0.000588,0.001610,-0.000232,-0.000103,-0.000627,0.001645,-0.000635


In [18]:
od = ordered_detector.values
od = np.log(od**2).transpose()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


In [19]:
@ipywidgets.interact(Time=(0,14400,1), Num=list(np.arange(0,438)))
def make_plot(Time, Num=0):
    fig = plt.figure(figsize=(12,8))
    # Plot the detectore on US map
    ax11 = plt.subplot2grid((12,8), (1,0), colspan=4, rowspan=6)
    ax11.scatter(location["longitude"],location["latitude"], c=detector.iloc[Time], cmap="Reds")
    # Reference; https://stackoverflow.com/questions/8202605/matplotlib-scatterplot-colour-as-a-function-of-a-third-variable
    ax11.set_title("Visualizing Tohoku Earthquake Detectors")
    ax11.axes.get_xaxis().set_visible(False)
    ax11.axes.get_yaxis().set_visible(False)
    # Reference: https://stackoverflow.com/questions/2176424/hiding-axis-text-in-matplotlib-plots
    ax21 = plt.subplot2grid((12,8), (0,5), colspan=3, rowspan=4)
    plt.imshow(od, aspect='auto', cmap='viridis', origin='lower')
    plt.plot([-100,15000],[Num, Num], 'k-', lw=1)
    plt.plot([Time, Time], [-10, 500], 'k-', lw=1)
    #y 轴坐标从上到下 0：437，需要调整
    ax21.patch.set_visible(False)
    ax21.axis('off')
    ax22 = plt.subplot2grid((12,8), (4,5), colspan=3, rowspan=3)
    ax22.plot(detector[Num])
    ax22.set_title(label = "Waveform of Detector No."+str(Num))
    plt.show()

interactive(children=(IntSlider(value=7200, description='Time', max=14400), Dropdown(description='Num', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437), value=0), Output()), _dom_classes=('widget-interact',))

# Audio

In [21]:
from IPython.display import Audio

In [25]:
# Reference: Week 10 class example
@ipywidgets.interact(detector=list(np.arange(0,438)))
def audio(detector):
    normed=(ordered_detector[detector]-ordered_detector[detector].min())/(ordered_detector[detector].max()-ordered_detector[detector].min())*2-1
    return Audio(normed, rate=14400/3, autoplay=False)   
  

interactive(children=(Dropdown(description='detector', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437), value=0), Output()), _dom_classes=('widget-interact',))